Сначала создадим матрицу, на которой будет обучение

In [2]:
import warnings
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import polars as pl
warnings.filterwarnings("ignore")
%matplotlib inline

In [55]:
df_logs = pl.scan_parquet(".\data\ml_ozon_recsys_train_final_apparel_tracker_data\part-00000-582d88ee-8dd6-41f6-9f90-375910468cb7-c000.snappy.parquet").select(["user_id", "item_id", "action_type"])
# df_logs = pl.scan_parquet(".\data\ml_ozon_recsys_train_final_apparel_tracker_data\*.parquet").select(["user_id", "item_id", "action_type"])

In [13]:
# aggregated_logs =  df_logs.with_columns(df_logs.select("action_type").collect().to_dummies()).drop("action_type").group_by(["user_id","item_id"]).sum()

In [56]:
df_orders = pl.scan_parquet(".\data\ml_ozon_recsys_train_final_apparel_orders_data\*.parquet")

In [57]:
df_orders.collect()["last_status"].unique()

last_status
str
"""delivered_orders"""
"""proccesed_orders"""
"""canceled_orders"""


In [15]:
# df_logs.select("action_type").collect().unique()

In [ ]:
aggregated_full_data = (df_logs
                        .join(df_orders,["item_id","user_id"], how="inner")
                        # .drop(['created_timestamp','last_status_timestamp','created_date'])
                        .filter("last_status" != "delivered_orders")
                        .fill_null(0))


aggregated_full_data = (aggregated_full_data.with_columns(
                                                (pl.col("action_type") == "view_description").cast(pl.Int8).alias("action_type_view_description"),
                                                (pl.col("action_type") == "to_cart").cast(pl.Int8).alias("action_type_to_cart"),
                                                (pl.col("action_type") == "page_view").cast(pl.Int8).alias("action_type_page_view"),
                                                (pl.col("action_type") == "favorite").cast(pl.Int8).alias("action_type_favorite"),
                                                (pl.col("action_type") == "unfavorite").cast(pl.Int8).alias("action_type_unfavorite"),
                                                (pl.col("action_type") == "review_view").cast(pl.Int8).alias("action_type_review_view"),
                                                (pl.col("action_type") == "remove").cast(pl.Int8).alias("action_type_remove"),
                                            )
                                            .drop(["action_type"])
                                            .group_by(["user_id","item_id"])
                                            .sum()
                                            .drop(["user_id","item_id"])
                                            .with_columns(
                                                pl.col("action_type_view_description").cast(pl.Int16),
                                                pl.col("action_type_to_cart").cast(pl.Int16),
                                                pl.col("action_type_page_view").cast(pl.Int16),
                                                pl.col("action_type_favorite").cast(pl.Int16),
                                                pl.col("action_type_unfavorite").cast(pl.Int16),
                                                pl.col("action_type_review_view").cast(pl.Int16),
                                                pl.col("action_type_remove").cast(pl.Int16),
                                            ))

In [9]:
n_rows = aggregated_full_data.select(pl.count()).collect().item()
n_rows

3832258

In [10]:
from tqdm import tqdm
batch_size = 1_000_000

for start in tqdm(range(0, n_rows, batch_size)):
    batch = aggregated_full_data.slice(start, batch_size).collect()
    batch.write_parquet(f".\\data\\train_log_reg_data\\batch_{start//batch_size}.parquet")

  0%|          | 0/4 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [9]:
aggregated_full_data.collect().write_parquet(".\\data\\train_log_reg_data\\data.parquet")

In [49]:
aggregated_full_data.collect()

created_timestamp,last_status,last_status_timestamp,created_date,action_type_view_description,action_type_to_cart,action_type_page_view,action_type_favorite,action_type_unfavorite,action_type_review_view,action_type_remove
datetime[ns],str,datetime[ns],str,i16,i16,i16,i16,i16,i16,i16
null,null,null,null,0,0,1,0,0,0,0
null,null,null,null,23,3,18,1,0,11,0
null,null,null,null,0,1,5,0,0,0,0
null,null,null,null,0,1,4,0,0,0,0
null,null,null,null,4,1,8,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,2,2,6,0,0,0,1
null,null,null,null,0,2,15,0,0,0,1
null,null,null,null,16,1,15,1,1,7,0


In [43]:
aggregated_full_data.collect()["last_status"].unique()

last_status
str
null
